# 🔬 Initialization & data wrangling

I denne blok sættes environmentet op, med initialisering af nødvendige libraries samt upload af data. Formålet er at skabe et udgangspunkt for data analyse med python og dets stærke data wrangling toolkits såsom pandas og numpy, data analysis vha. seaborn og kvantitativ analyse hjulpet af machine learning librariet scikit-learn 

In [5]:
# Indlæs libraries
import pandas as pd
import numpy as np

### 1. Indlæs og saml data
Indlæs de forskellige datasæt og saml dem i et.

In [15]:
# Filepaths, tilpas disse til lokal-sti

brancher_fp = r'/Users/albertcortbanke/Desktop/Case/data/.arbejdsmarkedsanalyse_brancher.csv.icloud'
koen_alder_fp = r'/Users/albertcortbanke/Desktop/Case/data/arbejdsmarkedsanalyse_koen_alder.csv'

In [16]:
# Indlæs data med pandas read_csv, brug delimiter = ; til at spalte data korrekt og samtidig enconding = 'latin-1' for at tilpasse encoding til dataet

branche_data = pd.read_csv(brancher_fp, delimiter = ";", encoding='latin-1')
koen_alder_data = pd.read_csv(koen_alder_fp, delimiter = ";", encoding='latin-1')

In [9]:
branche_data

,Field Values Index,Ordforklaring,Akse,Farveskala,Field Values Index (Fixed),Navigation - Arbejdsmiljøprofiler,Score (Indekseret score) (gennemsnit),Score (Indekseret score) (gennemsnit) (label),Score (Total) (Fixed),Gennemsnit,...,Main Group,Score,Field Values,Mean,Question Label,Sluttekst,Spm Formulering,Topic Label,Type,Year
0,"49,824544","Spørgsmål: Hvor ofte oplever du, at du har nok...",Score,"-5,059771093","54,88431509",Arbejdsmiljøprofiler,"52,18088073","2,609044037","-2,609044037",NaN,...,Job,"2,658698159","-2,658698159","2,658698159",Ikke nok tid til arbejdsopgaver,I resultaterne præsenteres den gennemsnitlige ...,"Hvor ofte oplever du, at du har nok tid til di...",Kvantitative krav og grænseløshed,kont,2018
1,"59,24585073","Spørgsmål: Hvor ofte oplever du, at du har nok...",Score,"4,361535635","54,88431509",Arbejdsmiljøprofiler,"52,18088073","2,609044037","-2,609044037",NaN,...,Job,"2,566242056","-2,566242056","2,566242056",Ikke nok tid til arbejdsopgaver,I resultaterne præsenteres den gennemsnitlige ...,"Hvor ofte oplever du, at du har nok tid til di...",Kvantitative krav og grænseløshed,kont,2018
2,"89,95423342","Spørgsmål: Hvor ofte oplever du, at du har nok...",Score,"35,06991833","54,88431509",Arbejdsmiljøprofiler,"52,18088073","2,609044037","-2,609044037",NaN,...,Job,"2,264884984","-2,264884984","2,264884984",Ikke nok tid til arbejdsopgaver,I resultaterne præsenteres den gennemsnitlige ...,"Hvor ofte oplever du, at du har nok tid til di...",Kvantitative krav og grænseløshed,kont,2018
3,"44,36525953","Spørgsmål: Hvor ofte oplever du, at du har nok...",Score,"-10,51905556","54,88431509",Arbejdsmiljøprofiler,"52,18088073","2,609044037","-2,609044037",NaN,...,Job,"2,712272911","-2,712272911","2,712272911",Ikke nok tid til arbejdsopgaver,I resultaterne præsenteres den gennemsnitlige ...,"Hvor ofte oplever du, at du har nok tid til di...",Kvantitative krav og grænseløshed,kont,2018
4,"85,4477087","Spørgsmål: Hvor ofte oplever du, at du har nok...",Score,"30,5633936","54,88431509",Arbejdsmiljøprofiler,"52,18088073","2,609044037","-2,609044037",NaN,...,Job,"2,309109817","-2,309109817","2,309109817",Ikke nok tid til arbejdsopgaver,I resultaterne præsenteres den gennemsnitlige ...,"Hvor ofte oplever du, at du har nok tid til di...",Kvantitative krav og grænseløshed,kont,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8110,"49,8865146","Spørgsmål: Forestil dig, at din arbejdsevne er...",Score,0,"49,8865146",Arbejdsmiljøprofiler,"83,39333095","8,339333095","8,339333095","8,339333095",...,Job,"8,339333095","8,339333095","8,339333095",Arbejdsevne,\n\n I resultaterne præsenteres den gennemsni...,"Forestil dig, at din arbejdsevne er 10 point v...",Arbejdsevne,kont,2018
8111,"18,11586988","Spørgsmål: Forestil dig, at din arbejdsevne er...",Score,"-31,77064472","49,8865146",Arbejdsmiljøprofiler,"83,39333095","8,339333095","8,339333095",NaN,...,Job,"7,966818788","7,966818788","7,966818788",Arbejdsevne,\n\n I resultaterne præsenteres den gennemsni...,"Forestil dig, at din arbejdsevne er 10 point v...",Arbejdsevne,kont,2018
8112,"66,91324051","Spørgsmål: Forestil dig, at din arbejdsevne er...",Score,"17,02672591","49,8865146",Arbejdsmiljøprofiler,"83,39333095","8,339333095","8,339333095",NaN,...,Job,"8,53897333","8,53897333","8,53897333",Arbejdsevne,\n\n I resultaterne præsenteres den gennemsni...,"Forestil dig, at din arbejdsevne er 10 point v...",Arbejdsevne,kont,2018
8113,"68,80152033","Spørgsmål: Forestil dig, at din arbejdsevne er...",Score,"18,91500573","49,8865146",Arbejdsmiljøprofiler,"83,39333095","8,339333095","8,339333095",NaN,...,Job,"8,561113619","8,561113619","8,561113619",Arbejdsevne,\n\n I resultaterne præsenteres den gennemsni...,"Forestil dig, at din arbejdsevne er 10 point v...",Arbejdsevne,kont,2018
